In [ ]:


import earthaccess
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# setup & authentication
auth = earthaccess.login()

# hopkins bounds and full year 2024
bounds = {"min_lon": -120.75, "max_lon": -119, "min_lat": 33.5,, "max_lat": 34.5}
date_range = ("2024-01-01", "2024-12-31")

# create the master target grid
target_lat = np.arange(bounds['min_lat'], bounds['max_lat'], 0.01)
target_lon = np.arange(bounds['min_lon'], bounds['max_lon'], 0.01)
target_grid = xr.Dataset(coords={'lat': target_lat, 'lon': target_lon})

# fetch sst data
print("searching for mur sst data...")
sst_query = earthaccess.search_data(
    short_name="MUR-JPL-L4-GLOB-v4.1",
    bounding_box=(bounds['min_lon'], bounds['min_lat'], bounds['max_lon'], bounds['max_lat']),
    temporal=date_range
)

# open with decode_timedelta=True to fix the futurewarning
ds_sst = xr.open_mfdataset(
    earthaccess.open(sst_query), 
    chunks={'time': 30}, 
    decode_timedelta=True,
    engine="h5netcdf"
)


# we interp to the master grid to ensure future pace alignment
sst_celsius = (ds_sst.analysed_sst - 273.15).sel(
    lat=slice(bounds['min_lat'], bounds['max_lat']),
    lon=slice(bounds['min_lon'], bounds['max_lon'])
).interp_like(target_grid, method="linear")

# calculate monthly averages
print("calculating monthly means...")
sst_monthly = sst_celsius.resample(time="1MS").mean().compute()

# save to netcdf for use in your pace script
sst_monthly.to_netcdf("hopkins_sst_monthly_2024.nc")
print("baseline saved to 'hopkins_sst_monthly_2024.nc'")

# visualize the thermal year
# we use a 4x3 grid to show all 12 months of 2024
fig, axes = plt.subplots(4, 3, figsize=(15, 18))
axes = axes.flatten()

for i, m in enumerate(sst_monthly.time):
    m_str = pd.to_datetime(m.values).strftime('%B %Y')
    
    # plot using a 'thermal' colormap
    # vmin/vmax are set to highlight monterey bay temperature ranges
    im = sst_monthly.isel(time=i).plot(
        ax=axes[i], 
        cmap='RdYlBu_r', 
        vmin=10, 
        vmax=18, 
        add_colorbar=False
    )
    axes[i].set_title(m_str)
    axes[i].set_xlabel("")
    axes[i].set_ylabel("")

# add one common colorbar
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(im, cax=cbar_ax, label='temperature (°c)')

plt.suptitle("2024 monthly sea surface temperature", fontsize=16, y=0.95)
plt.show()